In [1]:
def matrix_factorization(R, P, Q, K, steps=5000, alpha=0.0002, beta=0.02):
    Q = Q.T
    for step in xrange(steps):
        for i in xrange(len(R)):
            for j in xrange(len(R[i])):
                if R[i][j] > 0:
                    eij = R[i][j] - numpy.dot(P[i,:],Q[:,j])
                    for k in xrange(K):
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        eR = numpy.dot(P,Q)
        e = 0
        for i in xrange(len(R)):
            for j in xrange(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - numpy.dot(P[i,:],Q[:,j]), 2)
                    for k in xrange(K):
                        e = e + (beta/2) * ( pow(P[i][k],2) + pow(Q[k][j],2) )
        if e < 0.001:
            break
    return P, Q.T


In [2]:
import numpy

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

tag_headers = ['user_id', 'movie_id', 'tag', 'timestamp']
tags = pd.read_table('C:/Users/Fizik/Downloads/coursera/nir/nir/movielens/tags.dat', sep='::', header=None, names=tag_headers)

rating_headers = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_table('C:/Users/Fizik/Downloads/coursera/nir/nir/movielens/ratings.dat', sep='::', header=None, names=rating_headers)

movie_headers = ['movie_id', 'title', 'genres']
movies = pd.read_table('C:/Users/Fizik/Downloads/coursera/nir/nir/movielens/movies.dat',
                       sep='::', header=None, names=movie_headers)
movie_titles = movies.title.tolist()

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators; you can avoid this warning by specifying engine='python'.
C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:9: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators; you can avoid this warning by specifying engine='python'.
C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators; you can avoid this warning by specifying engine='python'.


In [4]:
df = movies.join(ratings, on=['movie_id'], rsuffix='_r').join(tags, on=['movie_id'], rsuffix='_t')
del df['movie_id_r']
del df['user_id_t']
del df['movie_id_t']
del df['timestamp_t']

In [5]:
rp = df.pivot_table(columns=['movie_id'], index=['user_id'],values='rating')
rp = rp.fillna(0)
rp.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,65006,65011,65025,65027,65037,65088,65091,65126,65130,65133
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
R = rp.values
print R
print type(R)

[[ 5.  5.  5. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  3.  4.  4.]]
<type 'numpy.ndarray'>


In [9]:


R = numpy.array(R)

N = len(R)
M = len(R[0])
K = 2

P = numpy.random.rand(N,K)
Q = numpy.random.rand(M,K)
%timeit
nP, nQ = matrix_factorization(R, P, Q, K)

In [10]:
print nP
print nQ

[[ 2.85530691  2.44732609]
 [ 1.83234154  2.24926643]
 [ 2.28112323  2.53083776]
 [ 2.82046949  2.25416377]
 [ 2.38651419  2.58393783]
 [ 2.72160768  2.1239419 ]
 [ 2.34154029  2.6566023 ]
 [ 2.32870559  2.30855501]
 [ 2.95087856  1.84370775]
 [ 2.49302704  2.38779371]
 [ 2.23013129  2.84921746]
 [ 2.12803748  2.28337407]
 [ 2.30700946  2.25764093]
 [ 2.54907435  2.0204401 ]
 [ 1.92607765  2.70935612]
 [ 2.6913698   2.11948207]
 [ 2.51886856  2.14662951]
 [ 2.48665066  2.39998382]
 [ 1.95049856  2.51301492]
 [ 2.5513985   2.51951882]
 [ 2.58949252  2.33673951]
 [ 2.69087326  2.27568534]
 [ 2.00438038  2.16165082]
 [ 2.19575564  2.31954889]
 [ 1.37336155  2.30643932]
 [ 2.72151891  2.68691816]
 [ 2.2333998   1.99986797]
 [ 2.08647877  2.05793954]
 [ 1.97765478  2.53039471]
 [ 2.40516749  2.26035385]
 [ 1.9722708   2.42557029]
 [ 2.47134471  2.54061609]
 [ 2.74858998  2.09911289]
 [ 2.3577576   2.26860875]
 [ 2.20869324  2.4905139 ]
 [ 2.50170459  2.6803674 ]
 [ 2.33736377  3.1023556 ]
 

In [11]:
nR = numpy.dot(nP, nQ.T)
print nR

[[ 4.97831716  4.97832864  4.9784549  ...,  4.75525587  6.48213773
   6.27453662]
 [ 3.50329023  3.51874349  3.72277015 ...,  3.72091625  4.8690883
   5.00397893]
 [ 4.23889367  4.25200283  4.42507431 ...,  4.36666655  5.78018891
   5.84172893]
 ..., 
 [ 3.88136162  3.87599849  3.80516412 ...,  3.57716055  4.94685135
   4.6872646 ]
 [ 1.92833918  1.95880435  2.36105598 ...,  2.5807097   3.11737536
   3.59110865]
 [ 2.95865158  2.96544838  3.05517801 ...,  2.99076365  3.9875856
   3.98773105]]
